In [1]:
from data.get_voc_data import * 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from models import multilabel_cnn_classifier

Using TensorFlow backend.


In [2]:
classes = ['person', 'horse']

In [3]:
label_df = get_voc_labels(voc_path='/home/robin/Documents/VOCdevkit/VOC2012', classes=classes)

In [4]:
horse_and_person_df = label_df[(label_df==[1,1]).all(axis=1)]
horse_df = label_df[(label_df==[0,1]).all(axis=1)]
person_df = label_df[(label_df==[1,0]).all(axis=1)]

In [5]:
new_person_df = person_df.sample(horse_df.shape[0])

In [6]:
new_label_df = pd.concat([horse_df, new_person_df, horse_and_person_df])

In [7]:
new_label_df = shuffle(new_label_df)
images = get_voc_images(voc_path='/home/robin/Documents/VOCdevkit/VOC2012', image_type='reshaped', label_df=new_label_df)
labels = new_label_df.values

In [8]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [9]:
print(train_images[0].shape)
print(train_labels.shape)

(250, 250, 3)
(631, 2)


In [10]:
from data.get_data import get_fashion_mnist, get_cifar10
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import join as pathjoin
import os
from PIL import Image
from keras.optimizers import SGD
import ntpath

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=train_images[0].shape))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(len(classes), activation='sigmoid'))

 
opt = 'adam'
model.compile(optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy'])
model.summary()

model.fit(x=train_images, y=train_labels, batch_size=10, epochs=10, validation_split=0.1)
model.evaluate(x=test_images, y=test_labels, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 250, 250, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 250, 250, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 125, 125, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 125, 125, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)       

[1.0299225411837614, 0.642405092716217]

In [11]:
def eval(model, test_images, test_labels, classes):
        in_top1 = []
        in_top3 = []
        top_3_predictions = []
        for i in range(test_images.shape[0]):
            image = test_images[i]
            prediction = model.predict(np.asarray([image]))
            top_3 = [classes[j] for j in list(prediction[0].argsort()[-3:][::-1])]
            in_top1.append(int(np.argmax(test_labels[i])==np.argmax(prediction[0])))
            is_in_top3 = int(np.argmax(test_labels[i]) in list(prediction[0].argsort()[-3:][::-1]))
            in_top3.append(is_in_top3)
            top_3_predictions.append((is_in_top3, top_3, classes[np.argmax(test_labels[i])]))
        top1_score = sum(in_top1)/len(in_top1)
        top3_score = sum(in_top3)/len(in_top3)
        

            
        return top1_score, top3_score, top_3_predictions

In [12]:
top1_score, top3_score, top_3_predictions = eval(model, test_images, test_labels, classes)

In [13]:
print(top1_score)
print(top3_score)

0.5316455696202531
1.0


In [19]:
sum([ (element[1][0]==element[2]) for element in top_3_predictions])/len(top_3_predictions)

0.5316455696202531

In [20]:
word = 'voc_1234'
word[4:]

'1234'